# VBA Files

### Pulling Excel Files from the Website

In [ ]:
# There are weekly files from 2014 to present on https://www.benefits.va.gov/reports/detailed_claims_data.asp

# There are historical files at https://www.benefits.va.gov/REPORTS/mmwr/historical/{year}/index.asp from 2004 to 2013

# Presumably the first step will be just pulling all of them

### Libraries/Setup

In [10]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
os.chdir(r'C:\Users\admin\Documents\Python Scripts\veteranLawData')

### Get File URLs

In [61]:
def pullTheFiles(year):
    url= f'https://www.benefits.va.gov/REPORTS/mmwr/historical/{str(year)}/index.asp'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    filesToGet=[]
    for link in soup.findAll('a'):
        try:
            if "xls" in (link.get('href')):
                filesToGet.append(link.get('href'))
        except:
            pass
    return(filesToGet)

yearRange=list(range(2004, 2014))
#files=pullTheFiles(2004)
filesList=list(map(pullTheFiles, yearRange))
flatFileList = [item for sublist in filesList for item in sublist]

### Check We Have The Right Number: 2008 Is Legitimately Missing Three, I Checked

In [77]:
for list in filesList:
    print(len(list))

52
52
52
52
49
52
52
52
52
53


### Pull The Files

In [ ]:
def pullFile(url):
    cleanLink='https://www.benefits.va.gov'+url
    r = requests.get(cleanLink, allow_redirects=True)
    name=cleanLink.split("/")[-1]
    open(name, 'wb').write(r.content)
    return(name)

#listOfFileNames=list(map(pullFile, files))
for aFile in flatFileList:
    name=pullFile(aFile)
    print(name)

010504.xls
040504.xls
070504.xls
100404.xls
012004.xls
041204.xls
071204.xls
101204.xls
012004.xls
041904.xls
071904.xls
101804.xls
012604.xls
042604.xls
072604.xls
102504.xls
020204.xls
050304.xls
080204.xls
110104.xls
020904.xls
051004.xls
080904.xls
110804.xls
021704.xls
051704.xls
081604.xls
111504.xls
022304.xls
052404.xls
082304.xls
112204.xls
030104.xls
060104.xls
083004.xls
112904.xls
030804.xls
060704.xls
090604.xls
120604.xls
031504.xls
061404.xls
091304.xls
121304.xls
032204.xls
062104.xls
092004.xls
122004.xls
032904.xls
062804.xls
092704.xls
122704.xls
010305.xls
040405.xls
070505.xls
100305.xls
011005.xls
041105.xls
071105.xls
101105.xls
011805.xls
041805.xls
071805.xls
101705.xls
012405.xls
042505.xls
072505.xls
102405.xls
013105.xls
050205.xls
080105.xls
103105.xls
020505.xls
050905.xls
080805.xls
110705.xls
021205.xls
051605.xls
081505.xls
111405.xls
022205.xls
052305.xls
082205.xls
112105.xls
022805.xls
053105.xls
082605.xls
112805.xls
030705.xls
060405.xls
090605.xls

### Read Them Into A Dictionary

In [25]:
def readBasicFile(fileName):
    df=pd.read_excel(fileName)
    return(df)

listOfDFs=list(map(readBasicFile, listOfFileNames))
dictionaryOfFiles = dict(zip(listOfFileNames, listOfDFs))

### Append Into Master DF

In [57]:
masterDF=pd.DataFrame()
for key in dictionaryOfFiles.keys():
    df=dictionaryOfFiles[key]
    df['fileName']=key
    listOfPhrasesToFind=["As of", "For the Week"]
    foundPhrase=[i for i in df.columns if any(substring in i for substring in listOfPhrasesToFind)]
    dropPhrase=[i for i in df.columns if "Percent" in i]
    #print(len(foundPhrase))
    df["forTheWeek"]=df[foundPhrase[0]]
    df=df.drop([foundPhrase[0]], axis=1)
    df=df.drop(dropPhrase, axis=1)
    masterDF=masterDF.append(df, sort=True)

In [59]:
masterDF.columns

Index(['C&P Claims over 180 Days in WIPP', 'COEs Issued',
       'Date of Oldest Pending COE', 'Education Cases Pending ',
       'Education Work Items Pending ', 'Guarantees Pending',
       'IVMs Pending (EP 154 and EP 314)',
       'Non-Rating Cases Pending over 180 Days', 'Pre-Discharge Claims',
       'Rating Cases Pending over 180 Days', 'SOC's',
       'Scorecard Non-Rating Cases Pending', 'Scorecard Rating Cases Pending',
       'Total Appeals Requiring Adjudicative Action ',
       'Total Appeals Requiring Adjudicative action ',
       'Total C&P Pending in Wipp', 'VACOLS Appeals',
       'VACOLS Appeals includes AMC & Travel Board', 'fileName', 'forTheWeek'],
      dtype='object')